# Prerrequisites

Installing Spark


---



In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip -q install findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# create the session
spark = SparkSession \
        .builder \
        .master("local[*]") \
        .getOrCreate()

spark.version

Creating tunnel</br>
**To Check the Spark UI, open the URL printed by running the above command : https://######/jobs/, /SQL/**


In [ ]:
 from google.colab.output import eval_js
 print(eval_js("google.colab.kernel.proxyPort(4040)") + "jobs/")

# Download Datasets

In [ ]:
!mkdir -p /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/bank.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/vehicles.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/characters.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/planets.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/species.csv -P /dataset
!ls /dataset

# Windows Partitioning

---



## Example 1

In [ ]:
!head /dataset/bank.csv

Reading data from `bank.csv` file to a DataFrame

In [ ]:
from pyspark.sql.functions import *

bank_df = spark.read.format("csv") \
  .option("sep", ";") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load("/dataset/bank.csv")

In [ ]:
bank_df.show()

Get the balance of the two youngest people by job


In [ ]:
from pyspark.sql.window import Window

byJob = Window.partitionBy("job").orderBy("age")

bank_df \
  .withColumn("new_column_job", row_number().over(byJob)) \
  .filter(col("new_column_job") <= 2) \
  .select("age", "job", "balance") \
  .orderBy("job", "age") \
  .show()

## Exercise 1

Using the dataframe built from `bank.csv`file, get the TOP 3 of maximum balance by marital
Obtén el Top 3 de máximos balances por estado civil


---




## Exercise 2



Load `vehicles.csv` file into a DataFrame

---

In [ ]:
!head /dataset/vehicles.csv

For each vehicle, get the difference in price (`cost_in_credits`) for each product compared to the cheapest product in the same vehicle class


---



# Joins

## Exercise 3

1. Create dataframes for files `characters.csv` and `planets.csv`
2. Get the planet gravity for each character, selecting only the character name, planet name and gravity.


---




## Exercise 4

Check exercise 3. What join type are been used? Why?

---

After checking execution plan, execute the following instructions:

---

In [ ]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", '0')

In [ ]:
spark.conf.get("spark.sql.autoBroadcastJoinThreshold")

Execute again the query of the exercise 3

---

## Exercise 5

1. Create a DataFrame from `species.csv`.
2. Repartition the previous DataFrame to 100 partitions
3. Repartition `characters` DataFrame to 100 partitions

---



## Exercise 6

Get the specie classification for each character. Select only the character name and its classification<br>
Use DataFrames repartitioned previously


---



## Exercise 7

1. Execute the following statement over the DataFrame built in exercise 6
2. Check the difference in terms of rows distribution across all partitions

---



In [ ]:
from pyspark.sql.functions import *

classDF \
  .withColumn("partitionId", spark_partition_id()) \
  .groupBy("partitionId") \
  .count() \
  .orderBy(col("count").desc()) \
  .show()